# 1. 环境设置

## 1.1 设置随机数种子

In [1]:
import torch
import numpy as np
import random

seed=2027
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)            # 为CPU设置随机种子
torch.cuda.manual_seed(seed)       # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)   # 为所有GPU设置随机种子

## 1.2 设置超参数

In [2]:
import os
trm_num = 2              # trm层数
trm_head_num = 4         # trm头数
trm_hidden_size = 16     # 隐藏层单元数（即历史行为单个token的embed维度）
trm_overlay = True       # 使用多个trm叠加（而不是单个循环）
maxlen = 20              # 序列最大长度
behavior_embed = 16      # 用户行为特征的embedding维度
file = 'data/amazon/beauty/beauty_remap_py36.pkl' # 评分文件

device = 'cuda'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

cuda ready...


## 1.3 工具函数（保存hist）

In [3]:
import pandas as pd

def save_history(history, path):
    hist = pd.DataFrame(history.history)
    hist.to_csv(path)

## 1.4 创建文件夹

In [4]:
basepath = 'hist/' + str(seed) + '/amazon_beauty_pretrain_epochs_log_' + str(seed)
histpath = basepath + '/fine'
prepath = basepath + '/pre'

if not os.path.exists(basepath):   # 如果历史文件夹不存在，先创建
    os.mkdir(basepath)
    os.mkdir(histpath)
    os.mkdir(prepath) 

# 2. 实验部分

## 2.1 Trm4Rec数据生成

In [5]:
import torch
import pickle
from deepctr_torch.process.pretrain_data_precess import create_sequence_amazon_movies_dataset

sequence_file = basepath + '/sequence_data.pkl'
if os.path.exists(sequence_file):
    with open(sequence_file, 'rb') as f:
        x, y, feature_columns, behavior_feature_list, item_id_max, pretrain_fea_col = pickle.load(f)
else:        
    # 生成微调阶段数据
    x, y, feature_columns, behavior_feature_list, item_id_max, pretrain_fea_col = create_sequence_amazon_movies_dataset(
        file, maxlen, behavior_embed, masked=True, pretrain=True
    )
    with open(sequence_file, 'wb') as f:
        pickle.dump(
            (x, y, feature_columns, behavior_feature_list, item_id_max, pretrain_fea_col),
            f,
            pickle.HIGHEST_PROTOCOL
        )

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## 2.2 DRIC数据生成

In [6]:
from deepctr_torch.process.pretrain_data_precess import create_delete_random_dataset
from deepctr_torch.models.trm4rec import Trm4Rec

dric_file = basepath + '/dric_data.pkl'

prob = [0.1, 0.25]
classify=4

if os.path.exists(dric_file):
    with open(dric_file, 'rb') as f:
        dr_sequences, dr_seq_len, dr_labels = pickle.load(f)
else:
    dr_sequences, dr_seq_len, dr_labels = create_delete_random_dataset(
        x['hist_item_id'], x['seq_length'], maxlen, item_id_max, prob, classify=4
    )
    
    with open(dric_file, 'wb') as f:
        pickle.dump(
            (dr_sequences, dr_seq_len, dr_labels),
            f,
            pickle.HIGHEST_PROTOCOL
        )

dr_item_sequence = {
    'hist_item_id': dr_sequences,
    'seq_length': dr_seq_len
}

## 2.3 创建DRIC模型

In [7]:
from deepctr_torch.models.dric import DRIC
from torch.optim import Adam

print('------创建DRIC------')

dric_model = DRIC(
    pretrain_fea_col,
    behavior_feature_list,
    seq_max_len=maxlen,
    device=device,
    classify=4,
    trm_num=2,
    trm_head_num=4,
    trm_overlay=True,
    trm_hidden_size=behavior_embed,
)

dric_model.compile(
    Adam(dric_model.parameters(), lr=0.002),
    'crossentropy',
    metrics=['crossentropy']
)

------创建DRIC------


## 2.4 预训练50轮
每10轮保存一次权重

In [10]:
dric_hist= dric_model.fit(
    dr_item_sequence,
    dr_labels,
    batch_size=2048,
    epochs=10,
    verbose=2,
    validation_split=0.2,
    save_weight=2,
    save_path=prepath + '/dric_weight_epoch_'
)

save_history(dric_hist, prepath + '/dric_hist.csv')

cuda:0
Train on 783886 samples, validate on 195972 samples, 383 steps per epoch
Epoch 1/10
33s - loss:  0.0314
Epoch 2/10
35s - loss:  0.0289
Epoch 3/10
48s - loss:  0.0277
Epoch 4/10
56s - loss:  0.0272
Epoch 5/10
57s - loss:  0.0266
Epoch 6/10
57s - loss:  0.0258
Epoch 7/10
56s - loss:  0.0253
Epoch 8/10
59s - loss:  0.0249
Epoch 9/10
57s - loss:  0.0244
Epoch 10/10
57s - loss:  0.0241


In [12]:
for i in range(1, 6):
    from deepctr_torch.models.trm4rec import Trm4Rec
    print('------创建Trm4Rec------')

    dr_fine_model = Trm4Rec(
        feature_columns,
        behavior_feature_list,
        maxlen,
        device=device,
        trm_num=2,
        trm_head_num=4,
        trm_overlay=trm_overlay,
        trm_hidden_size=behavior_embed,
        att_weight_normalization=True
    )

    print('------加载权重------')
    # 读取预训练得到的权重
    dr_pretrain_weight = torch.load(prepath + '/dric_weight_epoch_' + str(i*2) + '.pth')
    # 获取微调模型的权重
    dr_fine_weight = dr_fine_model.state_dict()
    # 只保留重合部分的权重
    model_weight = {
        k: v
        for k, v in dr_pretrain_weight.items()
        if k in dr_fine_weight
    }

    # 更新微调模型的权重字典
    dr_fine_weight.update(model_weight)
    # 加载权重
    dr_fine_model.load_state_dict(dr_fine_weight)

    dr_fine_model.compile(
        'adam',
        'binary_crossentropy',
        metrics=['auc', 'binary_crossentropy']
    )

    dr_fine_history = dr_fine_model.fit(
        x, y,
        batch_size=2048,
        epochs=5,
        verbose=2,
        validation_split=0.2
    )

    save_history(dr_fine_history, histpath + '/Trm4Rec' + str(i*2) + '.csv')

------创建Trm4Rec------
------加载权重------
cuda:0
Train on 1300478 samples, validate on 325120 samples, 635 steps per epoch
Epoch 1/5
129s - loss:  0.4914 - auc:  0.8413 - binary_crossentropy:  0.4914 - val_auc:  0.8543 - val_binary_crossentropy:  0.4681
Epoch 2/5
126s - loss:  0.4044 - auc:  0.8976 - binary_crossentropy:  0.4044 - val_auc:  0.8451 - val_binary_crossentropy:  0.4903
Epoch 3/5
127s - loss:  0.3105 - auc:  0.9390 - binary_crossentropy:  0.3105 - val_auc:  0.8294 - val_binary_crossentropy:  0.6049
Epoch 4/5
127s - loss:  0.1989 - auc:  0.9745 - binary_crossentropy:  0.1989 - val_auc:  0.7994 - val_binary_crossentropy:  0.8556
Epoch 5/5
145s - loss:  0.1104 - auc:  0.9923 - binary_crossentropy:  0.1104 - val_auc:  0.7826 - val_binary_crossentropy:  1.2051
------创建Trm4Rec------
------加载权重------
cuda:0
Train on 1300478 samples, validate on 325120 samples, 635 steps per epoch
Epoch 1/5
128s - loss:  0.4892 - auc:  0.8426 - binary_crossentropy:  0.4892 - val_auc:  0.8564 - val_bin